In [1]:
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 614.1 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e0873aeec4ab758d4465f61e01f28dc072ebc8f1031d794bc47091ab11b56cb9
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.2 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


In [2]:
import datasets
import numpy as np

In [3]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

train_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'train[:20000]')
validation_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'validation')
test_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'test')



Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


In [4]:
features = train_dataset.features
print(features)

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [5]:
text_column_name = "tokens"
label_column_name = "ner_tags"

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [6]:
# Defining Base Model and tokenizer
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
config = AutoConfig.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER", num_labels=num_labels)

2024-03-13 10:46:25.272528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 10:46:25.272627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 10:46:25.600964: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
# Tokenize all texts and align the labels with them.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding="max_length",
        truncation=True,
        max_length=256,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    # print(tokenized_inputs)
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    # print(tokenized_inputs)
    return tokenized_inputs

In [8]:
correct_output = tokenize_and_align_labels(train_dataset[0:1])
tokens = tokenizer.convert_ids_to_tokens(correct_output["input_ids"][0])
print(tokens)

# Now we have input ids and labels to pass to model

['[CLS]', 'सक', '##टर', '55', '/', '56', 'क', 'एस', '##ए', '##च', '##ओ', 'अ', '##र', '##वि', '##द', 'कमार', 'न', 'ब', '##ता', '##या', 'कि', 'इस', 'मा', '##मल', 'म', 'आई', '##पी', '##सी', 'की', 'ध', '##ारा', '376', '-', 'डी', '(', 'ग', '##गर', '##प', ')', 'क', 'त', '##हत', 'मा', '##मल', '##ा', 'दर', '##ज', 'कर', 'लिया', 'गया', 'ह', '।', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [9]:
# Tokenizing Train Dataset
tokenized_train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_train_dataset)

  0%|          | 0/20 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})


In [10]:
# Problem of uneven length is fixed now!!
print(len(tokenized_train_dataset[0]['ner_tags']))
print(len(tokenized_train_dataset[0]['labels']))
print(len(tokenized_train_dataset[0]['input_ids']))

30
256
256


In [11]:
# Tokenizing Test Dataset
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_test_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 867
})


In [12]:
# Tokenizing Validation DataSet
tokenized_validation_dataset = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_validation_dataset)

  0%|          | 0/14 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 13460
})


In [13]:
# Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [14]:
# Metric to see how my model is performing For NER we use seqeval
metric = datasets.load_metric("seqeval")

In [15]:
# Seeing the working of seqeval
example_text = train_dataset[0]
label_list = train_dataset.features["ner_tags"].feature.names
label_list

labels = [label_list[i] for i in example_text["ner_tags"]]
print(labels)

metric.compute(predictions=[labels] , references=[labels])

['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


{'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [16]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [24]:
# Define Training Args
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    learning_rate=1e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
)

In [25]:
trainer = Trainer(
   model,
   args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_validation_dataset,
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [26]:
train_result = trainer.train()
metrics = train_result.metrics

Step,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
500,0.123000,0.188716,0.810007,0.852835,0.830869,10213,0.675689,0.683630,0.679636,9786,0.789464,0.838096,0.813054,10568,0.761052,0.793568,0.776970,0.946845
1000,0.112800,0.190316,0.816113,0.846079,0.830825,10213,0.673960,0.695586,0.684602,9786,0.803872,0.836961,0.820083,10568,0.766558,0.794746,0.780398,0.947427
1500,0.101900,0.193905,0.822379,0.844120,0.833108,10213,0.679375,0.688330,0.683823,9786,0.803852,0.837245,0.820209,10568,0.770737,0.791867,0.781159,0.947466
2000,0.111100,0.190029,0.816919,0.851953,0.834068,10213,0.677025,0.690170,0.683534,9786,0.802688,0.836488,0.819239,10568,0.767857,0.794811,0.781102,0.947374


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory model/upos/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [54]:
for key,value in metrics.items():
    print(key,value)

train_runtime 1824.964
train_samples_per_second 21.918
train_steps_per_second 1.096
total_flos 5226171371520000.0
train_loss 0.7779098052978516
epoch 2.0


In [55]:
# Macro F1
sumoff1score=metrics['eval_LOC_f1']+metrics['eval_ORG_f1']+metrics['eval_PER_f1']
macrof1=sumoff1score/3
print("Macro F1 : " ,macrof1)

KeyError: 'eval_LOC_f1'

In [ ]:
# Save Model
trainer.save_model("model_indic_ner_5Epoch_one_lakh")

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    ___
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    ___
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)
zip_dir()

In [ ]:
# Evaluading Model
for key, value in metrics.items():
    print(f"{key}: {value}")

# Testing on 20% Testing Dataset

In [29]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode
lang = 'hi'
test_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'test')
# Test Dataset
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_test_dataset)

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 867
})


In [30]:
# Evaluation of Model on test Data
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)
trainer.log_metrics("Test",metrics)

sum_of_f1_scores=metrics['test_LOC_f1']+metrics['test_ORG_f1']+metrics['test_PER_f1']
macro_f1=sum_of_f1_scores/3
print('\n Macro f1 score::',macro_f1)

***** Test metrics *****
  test_LOC_f1             =     0.8039
  test_LOC_number         =        614
  test_LOC_precision      =        0.8
  test_LOC_recall         =     0.8078
  test_ORG_f1             =     0.7088
  test_ORG_number         =        525
  test_ORG_precision      =     0.6502
  test_ORG_recall         =      0.779
  test_PER_f1             =     0.8697
  test_PER_number         =        790
  test_PER_precision      =     0.8414
  test_PER_recall         =        0.9
  test_loss               =     0.1564
  test_overall_accuracy   =     0.9549
  test_overall_f1         =     0.8034
  test_overall_precision  =     0.7717
  test_overall_recall     =     0.8377
  test_runtime            = 0:00:10.36
  test_samples_per_second =     83.639
  test_steps_per_second   =      4.245

 Macro f1 score:: 0.7941511271452497
